In [3]:
import os
import sys
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

PROJECT_ROOT = r"D:\demand_forecasting_system"
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

from tasks.extract_mssql import fetch_table_data

In [4]:
df_orders = fetch_table_data("blinkit_orders")

D:\demand_forecasting_system\src\tasks\extract_mssql.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
2025-10-21 21:06:10,625 | INFO | data_pipeline | Fetched 5000 rows from table 'blinkit_orders' (DB)


In [3]:
df_orders.head()

,order_id,customer_id,order_date,promised_delivery_time,actual_delivery_time,delivery_status,order_total,payment_method,delivery_partner_id,store_id
0,60465,15808945,2024-10-23 05:23:29,2024-10-23 05:34:29,2024-10-23 05:39:29,On Time,589.469971,UPI,58998,3943
1,2237858,48281892,2023-04-02 03:45:11,2023-04-02 04:00:11,2023-04-02 04:03:11,On Time,3835.260010,Wallet,97945,1987
2,3101265,89617089,2024-05-23 03:21:47,2024-05-23 03:37:47,2024-05-23 03:40:47,On Time,3567.729980,UPI,20246,974
3,5120698,44174426,2023-06-09 12:10:20,2023-06-09 12:25:20,2023-06-09 12:40:20,Slightly Delayed,3064.360107,Wallet,48582,3184
4,5512907,51476157,2023-04-30 20:52:21,2023-04-30 21:11:21,2023-04-30 21:26:21,Slightly Delayed,861.530029,Wallet,44545,419


In [4]:
df_orders=df_orders[[ "order_id", "customer_id", "order_date","order_total", "payment_method", "store_id","promised_delivery_time", "actual_delivery_time", "delivery_status"]]

In [5]:
print("Columns in your DataFrame:\n", df_orders.columns.tolist())

Columns in your DataFrame:
 ['order_id', 'customer_id', 'order_date', 'order_total', 'payment_method', 'store_id', 'promised_delivery_time', 'actual_delivery_time', 'delivery_status']


In [6]:
df_orders.info()
df_orders.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   order_id                5000 non-null   int64         
 1   customer_id             5000 non-null   int64         
 2   order_date              5000 non-null   datetime64[ns]
 3   order_total             5000 non-null   float64       
 4   payment_method          5000 non-null   object        
 5   store_id                5000 non-null   int64         
 6   promised_delivery_time  5000 non-null   datetime64[ns]
 7   actual_delivery_time    5000 non-null   datetime64[ns]
 8   delivery_status         5000 non-null   object        
dtypes: datetime64[ns](3), float64(1), int64(3), object(2)
memory usage: 351.7+ KB


,order_id,customer_id,order_date,order_total,payment_method,store_id,promised_delivery_time,actual_delivery_time,delivery_status
0,60465,15808945,2024-10-23 05:23:29,589.469971,UPI,3943,2024-10-23 05:34:29,2024-10-23 05:39:29,On Time
1,2237858,48281892,2023-04-02 03:45:11,3835.260010,Wallet,1987,2023-04-02 04:00:11,2023-04-02 04:03:11,On Time
2,3101265,89617089,2024-05-23 03:21:47,3567.729980,UPI,974,2024-05-23 03:37:47,2024-05-23 03:40:47,On Time
3,5120698,44174426,2023-06-09 12:10:20,3064.360107,Wallet,3184,2023-06-09 12:25:20,2023-06-09 12:40:20,Slightly Delayed
4,5512907,51476157,2023-04-30 20:52:21,861.530029,Wallet,419,2023-04-30 21:11:21,2023-04-30 21:26:21,Slightly Delayed


In [7]:
df_orders.drop_duplicates(subset="order_id", keep="first", inplace=True)


In [8]:
# 3️⃣ Handle missing values (pandas 3.0 safe)
df_orders['payment_method'] = df_orders['payment_method'].fillna('Unknown')
df_orders['delivery_status'] = df_orders['delivery_status'].fillna('Pending')
df_orders['actual_delivery_time'] = df_orders['actual_delivery_time'].fillna(df_orders['promised_delivery_time'])



In [9]:
df_orders['order_date'] = pd.to_datetime(df_orders['order_date'], errors='coerce')
df_orders['promised_delivery_time'] = pd.to_datetime(df_orders['promised_delivery_time'], errors='coerce')
df_orders['actual_delivery_time'] = pd.to_datetime(df_orders['actual_delivery_time'], errors='coerce')


In [10]:

df_orders['payment_method'] = df_orders['payment_method'].str.title().replace({
    'Cod': 'COD', 
    'Cc': 'Credit Card', 
    'Dc': 'Debit Card'
})

df_orders['delivery_status'] = df_orders['delivery_status'].str.strip().str.title()


In [11]:
Q1 = df_orders['order_total'].quantile(0.25)
Q3 = df_orders['order_total'].quantile(0.75)
IQR = Q3 - Q1

upper_limit = Q3 + 1.5 * IQR
lower_limit = Q1 - 1.5 * IQR

# Filter or cap
df_orders = df_orders[(df_orders['order_total'] >= lower_limit) & (df_orders['order_total'] <= upper_limit)]


In [12]:
df_orders['delivery_delay_hrs'] = (
    (df_orders['actual_delivery_time'] - df_orders['promised_delivery_time'])
    .dt.total_seconds() / 3600
)


In [13]:
print(df_orders.isnull().sum())
print(df_orders.dtypes)
print(df_orders.shape)


order_id                  0
customer_id               0
order_date                0
order_total               0
payment_method            0
store_id                  0
promised_delivery_time    0
actual_delivery_time      0
delivery_status           0
delivery_delay_hrs        0
dtype: int64
order_id                           int64
customer_id                        int64
order_date                datetime64[ns]
order_total                      float64
payment_method                    object
store_id                           int64
promised_delivery_time    datetime64[ns]
actual_delivery_time      datetime64[ns]
delivery_status                   object
delivery_delay_hrs               float64
dtype: object
(4997, 10)


In [14]:
import os

# Define target path
save_path = r"D:\demand_forecasting_system\data\processed\blinkit_orders_clean.csv"

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the cleaned DataFrame
df_orders.to_csv(save_path, index=False)

print(f"Cleaned data saved successfully at: {save_path}")


Cleaned data saved successfully at: D:\demand_forecasting_system\data\processed\blinkit_orders_clean.csv
